# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [ ]:
#!pip install numpy
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [14]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [2]:
# load the dataset
path= "den_eigen_10.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13,DEN_EIGEN14
0,1,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.000000,-0.414214,-1.674492e-01,0.119329,2.414214,2.528918,2.886967,5.185428
1,1,-2.491632,-2.000000,-2.000000,-2.000000,-2.000000,-1.112333,-0.732051,-0.554352,-1.215415e-15,0.082645,2.207494,2.732051,2.835635,5.032543
2,1,-2.361469,-2.314888,-2.000000,-2.000000,-1.772638,-1.062436,-1.000000,-0.353747,-1.674492e-01,0.248641,2.371348,2.528918,2.827518,5.056201
3,1,-2.583876,-2.183713,-2.000000,-2.000000,-1.720551,-1.078207,-0.685097,-0.245004,-2.142153e-01,0.167653,2.207130,2.538845,2.870232,4.926804
4,1,-2.618034,-2.327191,-2.018542,-1.860806,-1.703671,-1.000000,-0.381966,-0.254102,-2.117450e-01,0.085637,2.114908,2.444219,2.945225,4.786068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153247,0,-2.811227,-2.601950,-1.885875,-1.730640,-1.484764,-1.253520,-0.747326,0.056430,2.309001e-01,0.778453,0.972495,1.379590,3.669024,5.428411
153248,0,-2.860187,-2.512730,-1.826490,-1.773395,-1.499641,-1.250044,-0.455231,-0.060628,1.911881e-01,0.603253,1.054350,1.379588,3.798956,5.211010
153249,0,-2.783426,-2.576996,-1.982863,-1.668209,-1.435051,-1.244674,-0.920652,0.106454,2.726610e-01,0.890787,1.022913,1.496109,3.588454,5.234492
153250,0,-2.866843,-2.396396,-2.032638,-1.698872,-1.464497,-1.264781,-0.941908,0.244319,2.804478e-01,0.798675,1.216259,1.577589,3.530978,5.017669


In [3]:
# Let's round up the columns to 8 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 8 decimals
df[cols_to_round] = df[cols_to_round].round(8)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13,DEN_EIGEN14
0,1,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.000000,-0.414214,-0.167449,0.119329,2.414214,2.528918,2.886967,5.185428
1,1,-2.491632,-2.000000,-2.000000,-2.000000,-2.000000,-1.112333,-0.732051,-0.554352,-0.000000,0.082645,2.207494,2.732051,2.835635,5.032543
2,1,-2.361469,-2.314888,-2.000000,-2.000000,-1.772638,-1.062436,-1.000000,-0.353747,-0.167449,0.248641,2.371348,2.528918,2.827518,5.056201
3,1,-2.583876,-2.183713,-2.000000,-2.000000,-1.720551,-1.078207,-0.685097,-0.245004,-0.214215,0.167653,2.207130,2.538845,2.870232,4.926804
4,1,-2.618034,-2.327191,-2.018542,-1.860806,-1.703671,-1.000000,-0.381966,-0.254102,-0.211745,0.085637,2.114908,2.444219,2.945225,4.786068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153247,0,-2.811227,-2.601950,-1.885875,-1.730640,-1.484764,-1.253520,-0.747326,0.056430,0.230900,0.778453,0.972495,1.379590,3.669024,5.428411
153248,0,-2.860187,-2.512730,-1.826490,-1.773395,-1.499641,-1.250044,-0.455231,-0.060628,0.191188,0.603253,1.054350,1.379588,3.798956,5.211010
153249,0,-2.783426,-2.576996,-1.982863,-1.668209,-1.435051,-1.244674,-0.920652,0.106454,0.272661,0.890787,1.022913,1.496109,3.588454,5.234492
153250,0,-2.866843,-2.396396,-2.032638,-1.698872,-1.464497,-1.264781,-0.941908,0.244319,0.280448,0.798675,1.216259,1.577589,3.530978,5.017669


# ChatGpt clasifier computation

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [5]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [7]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7707415745000163
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.85      0.83     20398
           1       0.67      0.61      0.64     10253

    accuracy                           0.77     30651
   macro avg       0.74      0.73      0.74     30651
weighted avg       0.77      0.77      0.77     30651



### Decision tree

In [8]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7330592802844932
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80     20398
           1       0.60      0.61      0.60     10253

    accuracy                           0.73     30651
   macro avg       0.70      0.70      0.70     30651
weighted avg       0.73      0.73      0.73     30651



### Random forest

In [ ]:
# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=12)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.7959283547029461
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85     20398
           1       0.72      0.64      0.68     10253

    accuracy                           0.80     30651
   macro avg       0.77      0.76      0.76     30651
weighted avg       0.79      0.80      0.79     30651



In [10]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
        feature  importance
12  DEN_EIGEN13    0.158016
10  DEN_EIGEN11    0.116620
11  DEN_EIGEN12    0.116351
13  DEN_EIGEN14    0.088662


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [7]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.7690796857463524


In [ ]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


Polynomial fit

In [ ]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.7793296089385475


## Random forest  perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces an almost perfect score

In [11]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9998629707932033


In [12]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
         feature  importance
12  DEN_EIGEN13    0.159345
11  DEN_EIGEN12    0.116578
10  DEN_EIGEN11    0.111929
13  DEN_EIGEN14    0.089752
9   DEN_EIGEN10    0.064333
